In [0]:
%matplotlib inline
!pip install torch-geometric \
  torch-sparse==latest+cu101 \
  torch-scatter==latest+cu101 \
  torch-cluster==latest+cu101 \
  -f https://pytorch-geometric.com/whl/torch-1.5.0.html

!pip install torch-geometric \
  torch-sparse==latest+cu101 \
  torch-scatter==latest+cu101 \
  torch-cluster==latest+cu101 \
  -f https://pytorch-geometric.com/whl/torch-1.5.0.html

In [0]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
from functools import reduce
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data, DataLoader, InMemoryDataset

import numpy as np
import pandas as pd

from os import listdir
from os.path import isfile, join

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('running on '+ ("GPU" if torch.cuda.is_available() else "CPU"))

running on CPU


In [0]:
from google.colab import drive
drive.mount('/content/drive')
PATH = '/content/drive/My Drive/rubber'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
class Seq2SeqDataSet(InMemoryDataset):
    def __init__(self, root, input_sequence, output_sequence, transform=None, pre_transform=None):
        super(Seq2SeqDataSet, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_dir(self):
      if os.path.exists(self.root+PROCESSED_DIR):
        return self.root+'/cleaned'
      else:
        os.mkdir(self.root+PROCESSED_DIR)
        return self.root+'/cleaned'
        
    @property
    def processed_dir(self):
      if os.path.exists(self.root+PROCESSED_DIR):
        return self.root+PROCESSED_DIR
      else:
        os.mkdir(self.root+PROCESSED_DIR)
        return self.root+PROCESSED_DIR

    @property
    def raw_file_names(self):
      mypath = self.raw_dir
      filenames = [f for f in listdir(mypath) if isfile(join(mypath, f))]
      return filenames

    @property
    def processed_file_names(self):
        return ['processed.dt']

    def download(self):
        pass
    
    def process(self):
      output_size , output_sequence_len, input_sequence_len= configuration['output_size'],\
      configuration['output_sequence_len'],\
      configuration['input_sequence_len'],
        
      data_list = []

      for raw_path in self.raw_paths:
        df = pd.read_csv(raw_path)
        for synthetic_seq in df['synthesis_seq'].unique():
          synthetic_data = df[df['synthesis_seq']==synthetic_seq]

          for country in synthetic_data['countryterritoryCode'].unique():
            country_data = synthetic_data[synthetic_data['countryterritoryCode'] == country]
                        
            del country_data['synthesis_seq']
            del country_data['countryterritoryCode']

            country_data_i = country_data[:-output_sequence_len]
            country_data_array = country_data_i.to_numpy()
            feature_length = len(country_data.columns)
            x = country_data_array#[:feature_length].T

            country_data_o = country_data[input_sequence_len:]
            #country_data_array = country_data_array.reshape(feature_length,len(country_data_i))
            country_data_array_y = np.array([country_data_o['NATURAL_RUBBER_EXPORT'].to_numpy()])
            country_data_array_y = country_data_array_y.reshape(output_size,len(country_data_o))
            y = country_data_array_y[:output_size].T

            #print(country_data_array.shape)
            #print(country_data_array[0])
            
            
            #print(x.shape)
            sets =0
            x_list = []
            dict_x = dict()
            for i in range(input_sequence_len):
              array_len = (len(x)-i) - ((len(x)  - i)%input_sequence_len)+i
              if array_len <= 0:
                print('skipping')
                continue
              sets = int( array_len/ input_sequence_len)
              if sets <= 0:
                print('skipping')
                continue
              #print(len(x))
              #print('input seq : ', i , ' ', array_len , ' ',array_len-i , ' number of sets : ', sets)
              x_temp = x[i:array_len].T.reshape(sets, feature_length, input_sequence_len)
              uniq_keys = np.array([i+k*input_sequence_len for k in range(sets)])
              x_temp = x_temp.reshape(feature_length,sets,input_sequence_len)
              arrays_split = np.hsplit(x_temp,sets)
              dict_x.update(dict(zip(uniq_keys, arrays_split)))

            dict_y = dict()
            y_list = []
            for i in range(output_sequence_len):
              array_len_y = (len(y)-i) - ((len(y)  - i)%output_sequence_len)+i
              if array_len_y <= 0:
                continue
              sets = int(array_len_y / output_sequence_len)
              if sets <= 0:
                continue
              y_temp = y[i:array_len_y].T.reshape(sets, output_size, output_sequence_len)
              #uniq_keys = np.array([i+(output_sequence_len*k) for k in range(output_sequence_len)])
              uniq_keys = np.array([i+k*output_sequence_len for k in range(sets)])
              y_temp = y_temp.reshape(output_size,sets,output_sequence_len)
              arrays_split = np.hsplit(y_temp,sets)
              dict_y.update(dict(zip(uniq_keys, arrays_split)))

            temp_x_list = []
            mean = np.mean(country_data.to_numpy(), axis=0).T
            std = np.std(country_data.to_numpy(), axis=0).T
            #print(sorted(dict_x.keys()))
            for i in sorted(dict_x.keys()):
              x = dict_x[i].squeeze()
              #print(x.T[0][12])
              x = (x.T - mean) / std
              where_are_NaNs = isnan(x)
              x[where_are_NaNs] = 0
              temp_x_list.append(x)

            temp_y_list  = [dict_y[i].T for i in sorted(dict_y.keys())]

            #_country_code,popData2018
            xy_list = [Data(x = torch.from_numpy(features).type(torch.FloatTensor).squeeze()) for features in temp_x_list]
            
            mean_y = np.mean(y)
            std_y = np.std(y)
            for i in sorted(dict_y.keys()):
              temp_y_list[i] = temp_y_list[i].squeeze()
              norm_y = (temp_y_list[i] -mean_y)/std_y
              xy_list[i].y = torch.from_numpy(norm_y).squeeze()

            data_list += xy_list
        print('processed : '+ raw_path)
      data, slices = self.collate(data_list)
      torch.save((data, slices), self.processed_paths[0])

In [0]:
#save this with the dataset notebook
configuration = {
    'input_sequence_len' : 6,
    'output_sequence_len' : 6,
    'training_batch_size' : 2048,
    'training_dataset_length' :32768,
    'validation_batch_size' : 1024,
    'yhat_size' : 1,
    'feature_len' : 21,
    'output_size' : 1,
}

INPUT_ROOT = PATH+'/input_mix'
CLEAN_DIR = 'cleaned_non_window'
DATA_TAG = "non_window_seq2seq_"+str(configuration['input_sequence_len'])+'_'+str(configuration['output_sequence_len'])
PROCESSED_DIR = '/processed_'+DATA_TAG

if os.path.exists(INPUT_ROOT+'/validation'+PROCESSED_DIR):
  for mypath in os.listdir(INPUT_ROOT+'/validation'+PROCESSED_DIR):
    os.remove(INPUT_ROOT+'/validation'+PROCESSED_DIR+'/'+mypath)

validation_dataset = CovidDataSet(INPUT_ROOT+'/training', configuration['input_sequence_len'], configuration['input_sequence_len'])

export = []
for i in range(len(validation_dataset)):
  export.append(validation_dataset[i].x.detach().numpy()[0][12])


validation_cleaned = pd.read_csv(INPUT_ROOT+'/validation/cleaned/DATA-25_seq.csv')
original_variable = np.array(validation_cleaned['OIL_PRICE'].tolist())
#print(export)
#print(original_variable)
original_variable = (original_variable - np.mean(original_variable, axis=0))/np.std(original_variable, axis=0)

orange_patch = mpatches.Patch(color='orange', label='Original')
blue_patch = mpatches.Patch(color='blue', label='After Processed')
plt.figure(figsize=(10,5))
plt.plot(original_variable)
plt.plot(export)

plt.legend(handles=[ blue_patch, orange_patch])
plt.show()

In [0]:
export = [0 for i in range(configuration['output_sequence_len'])]

for i in range(len(validation_dataset)):
  export.append(validation_dataset[i].y.detach().numpy()[0])

validation_cleaned = pd.read_csv(INPUT_ROOT+'/validation/cleaned/DATA-25_seq.csv')
original_export = np.array(validation_cleaned['NATURAL_RUBBER_EXPORT'].tolist())
original_export = (original_export - np.mean(original_export))/np.std(original_export)
plt.figure(figsize=(10,5))
plt.plot(original_export)
plt.plot(export)

orange_patch = mpatches.Patch(color='orange', label='Original')
blue_patch = mpatches.Patch(color='blue', label='After Processed')

plt.legend(handles=[ blue_patch, orange_patch])
plt.show()

In [0]:
#os.remove(INPUT_ROOT+"/processed/pre_transform.pt")
#os.remove(INPUT_ROOT+"/processed/processed.dt")
#os.remove(INPUT_ROOT+"/processed/pre_filter.pt")
#test dataset and loader
input_sequence_len = 6
output_sequence_len = 6
output_size = 1
feature_length = 23
INPUT_ROOT = PATH+'/input_mix/validation'
DATA_TAG = "seq2seq_5_5"
PROCESSED_DIR = '/processed_'+DATA_TAG

test_dataset = CovidDataSet(INPUT_ROOT, input_sequence_len, input_sequence_len)
print(test_dataset[0].x)

tensor([[1.9452e+08, 2.6121e+01, 8.3100e+01, 1.6300e+00, 1.0000e+00, 1.0230e+04,
         2.6300e+00, 4.9500e+02, 0.0000e+00, 8.7020e+03, 3.7600e+02, 2.9700e+02,
         2.5000e+01, 9.6705e+03, 2.5240e+02, 1.8350e+02, 0.0000e+00, 1.2925e+04,
         1.3850e+02, 2.3800e+02, 2.1692e+02, 6.1190e+01, 1.4240e+02],
        [2.6550e+02, 2.6110e+02, 5.3970e+01, 3.2775e+02, 1.3154e+08, 2.6550e+01,
         1.0746e+02, 5.5000e-01, 1.0000e+00, 8.5610e+03, 5.2500e+00, 2.7100e+02,
         0.0000e+00, 9.7495e+03, 2.2500e+02, 1.8350e+02, 2.5000e+01, 1.2685e+04,
         1.3900e+02, 2.8500e+02, 7.0000e+00, 1.3352e+04, 1.5400e+02],
        [2.6025e+02, 0.0000e+00, 9.3925e+03, 2.9700e+02, 2.8850e+02, 1.9379e+02,
         8.1720e+01, 3.0800e+02, 1.9978e+08, 2.6415e+01, 9.8140e+01, 4.1000e-01,
         1.0000e+00, 9.6705e+03, 7.9400e+00, 1.6850e+02, 0.0000e+00, 1.2945e+04,
         1.2350e+02, 2.9280e+02, 2.1000e+01, 1.3465e+04, 1.6200e+02],
        [2.6725e+02, 2.5000e+01, 9.3755e+03, 3.2975e+02, 2.73

In [0]:
n= len(test_dataset)
print('possible batch sizes : ', set(reduce(list.__add__, ([i, n//i] for i in range(1, int(n**0.5) + 1) if n % i == 0))))
batch_size = 1

def data_set_is_correct():
  test_dataloader = DataLoader(test_dataset, batch_size)
  for batch in test_dataloader:
    inputs = np.array([
                       batch.x.view(batch_size, input_sequence_len, feature_length)[26+i][-1][0].item() for i in range(feature_length)
              ])
    output = batch.y.view(batch_size, output_sequence_len, output_size)[25+i][0:feature_length].T[0].detach().numpy()
    return np.sum(output-inputs) == 0

if data_set_is_correct() == True: print('Dataset is correct')
else: print('Corrupt dataset')

possible batch sizes :  {1, 11, 121}


IndexError: ignored